2.5. Uruchom klienta BigQuery z poziomu Pythona.

In [2]:
import os
import pandas as pd
from google.cloud import bigquery
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../windy-access-416710-5959edbfa345.json"  # lokalizacja pobranego klucza z punktu 1.4.
client = bigquery.Client()

 2.6. Pobierz dane do obiektu DataFrame.

In [3]:
query = "SELECT * FROM `bigquery-public-data.covid19_open_data.covid19_open_data` LIMIT 10"
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()

Część 4

Przeanalizuj poniższe przypadki. Zastanów się, jakie dane potrzebujesz do każdego z nich, a następnie zapisz je w osobnych, jak najprostszych obiektach DataFrame. Na tym etapie nie przetwarzaj ich. Zadbaj o czystość danych, m.in. usuń niepotrzebne puste wartości i duplikaty, zunifikuj sposób podawania informacji, zaproponuj sposób naprawy błędnych danych. Gotowe dane z obiektów DataFrame zapisz w osobnych plikach CSV.

4.1. Chcemy posiadać podstawowe dane o wszystkich krajach świata, które będą zrozumiałe dla człowieka oraz uniwersalne i potencjalnie przyszłościowe do dalszego przetwarzania.

In [26]:
query = "SELECT DISTINCT country_name, iso_3166_1_alpha_3, population,life_expectancy, area_sq_km, gdp_per_capita_usd FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE aggregation_level = 0 ORDER BY country_name"
query_country = client.query(query)
query_country_result = query_country.result()
country_df = query_country_result.to_dataframe()
country_df.dropna(inplace=True)
country_df.to_csv('wyniki\\country_data.csv', index=False)

4.2. Chcemy wygenerować statystyki dotyczące zachorowań na COVID-19 na całym świecie.

In [43]:
query = "SELECT country_name, date , new_confirmed, new_confirmed_age_0, new_confirmed_age_1, new_confirmed_age_2, new_confirmed_age_3, new_confirmed_age_4, new_confirmed_age_5, new_confirmed_age_6, new_confirmed_age_7, new_confirmed_age_8, new_confirmed_age_9 , cumulative_confirmed   FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE aggregation_level = 0 ORDER BY country_name "
query_disease = client.query(query)
query_disease_result = query_disease.result()
disease_df = query_disease_result.to_dataframe()
disease_df.to_csv('wyniki\\disease_data.csv', index=False)

KeyboardInterrupt: 

4.3. Chcemy poznać efekty COVID-19 poprzez uwypuklenie problemu śmiertelności ludzi spowodowanej wirusem.

In [44]:
query = "SELECT country_name, date, new_deceased , new_deceased_age_0 , new_deceased_age_1 , new_deceased_age_2 , new_deceased_age_3 , new_deceased_age_4 , new_deceased_age_5 , new_deceased_age_6 , new_deceased_age_7 , new_deceased_age_8 , new_deceased_age_9, cumulative_deceased  FROM `bigquery-public-data.covid19_open_data.covid19_open_data`  WHERE aggregation_level = 0 ORDER BY country_name "
query_deceased = client.query(query)
query_deceased_result = query_deceased.result()
deceased_df = query_deceased_result.to_dataframe()
deceased_df.to_csv('wyniki\\deceased_data.csv', index=False)

4.4. Chcemy zaobserwować trendy i zależności dotyczące szczepień na COVID-19.

In [45]:
query = "SELECT country_name, date, new_persons_vaccinated, new_persons_fully_vaccinated, new_persons_fully_vaccinated_moderna, new_persons_fully_vaccinated_pfizer, new_persons_fully_vaccinated_janssen FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE aggregation_level = 0 ORDER BY country_name "
query_vaccine = client.query(query)
query_vaccine_result = query_vaccine.result()
vaccine_df = query_vaccine_result.to_dataframe()
vaccine_df.to_csv('wyniki\\vaccine_data.csv', index=False)

4.5. Zdefiniuj własny dodatkowy przypadek.

Liczba wykonanych testów na całym świecie

In [46]:
query = "SELECT country_name, date, new_tested, new_tested_age_0, new_tested_age_1, new_tested_age_2, new_tested_age_3, new_tested_age_4, new_tested_age_5, new_tested_age_6, new_tested_age_7, new_tested_age_8, new_tested_age_9, cumulative_tested FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE aggregation_level = 0 ORDER BY country_name "
query_tested = client.query(query)
query_tested_result = query_tested.result()
tested_df = query_tested_result.to_dataframe()
tested_df.to_csv('wyniki\\tested_data.csv', index=False)

Część 5

Połącz ze sobą wszystkie dane otrzymane w części 4. Nowy zbiór danych zapisz jako oddzielny obiekt DataFrame. Jeżeli uważasz, że należy stworzyć kilka takich obiektów, zrób to i zapisz swoje uzasadnienie. Pamiętaj o ciągłej konieczności zachowania czystości danych. Wynik tego zadania zapisz w pliku / plikach CSV.

In [48]:
merged_country_disease = pd.merge(country_df, disease_df, on=['country_name'])
merged_country_disease_deceased = pd.merge(merged_country_disease, deceased_df, on=['country_name'])
merged_country_disease_deceased_vaccine = pd.merge(merged_country_disease_deceased, vaccine_df, on=['country_name'])
merged_df_per_country = pd.merge(merged_country_disease_deceased_vaccine, tested_df, on=['country_name'])
merged_df_per_country.to_csv('wyniki\\part_5_merged.csv', index=False)

MemoryError: Unable to allocate 1.43 GiB for an array with shape (192487876,) and data type int64

Część 6

Połącz ze sobą dane otrzymane w części 5 oraz dane znajdujące się w plikach:
- world_countries.csv
- gdp.csv
Nowy zbiór danych zapisz jako oddzielny obiekt DataFrame. Jeżeli uważasz, że należy stworzyć kilka takich obiektów, zrób to i zapisz swoje uzasadnienie. Pamiętaj o ciągłej konieczności zachowania czystości danych. Wynik tego zadania zapisz w pliku / plikach CSV.

In [25]:
world_countries_df = pd.read_csv('dane_z_wikamp\\world_countries.csv')
gdp_df = pd.read_csv('dane_z_wikamp\\gdp.csv')

merged_df_per_country_world_countries = pd.merge(merged_df_per_country, world_countries_df, left_on='Country name', right_on='Country/Territory')
merged_df_per_country_world_countries.drop('Country/Territory', axis=1, inplace=True)
merged_df_per_country_world_countries.drop('CCA3', axis=1, inplace=True)
merged_df_per_country_world_countries.to_csv('wyniki\\part_6_01_merged.csv', index=False)

merged_df_data = pd.merge(merged_df_per_country_world_countries, gdp_df, left_on='Country name', right_on='Country Name')
merged_df_data.drop('Country Name', axis=1, inplace=True)
merged_df_data.drop('Country Code', axis=1, inplace=True)
merged_df_data.rename(columns={"Value" : "GDP Value"}, inplace=True)
merged_df_data.to_csv('wyniki\\part_6_02_merged.csv', index=False)